In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.linear_model import LinearRegression
import json
import csv
from sklearn import metrics
from tflite_benchmark_executor import TfLiteBenchmarkExecutor

In [2]:
with open('kernels.json') as json_file:
    kernels = json.load(json_file)

In [3]:
with open('squezzenet_layers.txt') as json_file:
    data = json.load(json_file)
    
    names = []
    for layer in data:
        names.append(layer)
        
    itens = []
    for name in names:
        itens.append(data[name])

In [4]:
    conv_2d = [[],[],[],[],[],[],[],[],[],[],[]]
    max_pol = [[],[],[],[],[],[],[],[],[],[],[]]
    
    final_test_conv_energy = []
    final_test_conv_time = []
    
    conv_2d_time = [[],[],[],[],[],[],[],[],[],[],[]]
    
    for kernel,layer in zip(kernels,itens):
        
        if(kernel["kernel"] == "CONV_2D"):
            
            aux = [layer["output"][0], layer["output"][1], layer["output"][2],
                  layer["filters"][0], layer["filters"][1], layer["filters"][2],
                  layer["stride"], layer["input"][0], layer["input"][1], layer["input"][2]]
            final_test_conv_energy.append(aux)
            final_test_conv_time.append(aux)
        
            conv_2d[0].extend(np.repeat(layer["output"][0], len(kernel['average_energy_per_image'])))
            conv_2d[1].extend(np.repeat(layer["output"][1], len(kernel['average_energy_per_image'])))
            conv_2d[2].extend(np.repeat(layer["output"][2], len(kernel['average_energy_per_image'])))
            conv_2d[3].extend(np.repeat(layer["filters"][0], len(kernel['average_energy_per_image'])))
            conv_2d[4].extend(np.repeat(layer["filters"][1], len(kernel['average_energy_per_image'])))
            conv_2d[5].extend(np.repeat(layer["filters"][2], len(kernel['average_energy_per_image'])))
            conv_2d[6].extend(np.repeat(layer["stride"], len(kernel['average_energy_per_image'])))
            conv_2d[7].extend(np.repeat(layer["input"][0], len(kernel['average_energy_per_image'])))
            conv_2d[8].extend(np.repeat(layer["input"][1], len(kernel['average_energy_per_image'])))
            conv_2d[9].extend(np.repeat(layer["input"][2], len(kernel['average_energy_per_image'])))
            conv_2d[10].extend(np.asarray(kernel['average_energy_per_image']))
    
            conv_2d_time[0].extend(np.repeat(layer["output"][0], len(kernel['average_time_per_image'])))
            conv_2d_time[1].extend(np.repeat(layer["output"][1], len(kernel['average_time_per_image'])))
            conv_2d_time[2].extend(np.repeat(layer["output"][2], len(kernel['average_time_per_image'])))
            conv_2d_time[3].extend(np.repeat(layer["filters"][0], len(kernel['average_time_per_image'])))
            conv_2d_time[4].extend(np.repeat(layer["filters"][1], len(kernel['average_time_per_image'])))
            conv_2d_time[5].extend(np.repeat(layer["filters"][2], len(kernel['average_time_per_image'])))
            conv_2d_time[6].extend(np.repeat(layer["stride"], len(kernel['average_time_per_image'])))
            conv_2d_time[7].extend(np.repeat(layer["input"][0], len(kernel['average_time_per_image'])))
            conv_2d_time[8].extend(np.repeat(layer["input"][1], len(kernel['average_time_per_image'])))
            conv_2d_time[9].extend(np.repeat(layer["input"][2], len(kernel['average_time_per_image'])))
            conv_2d_time[10].extend(np.asarray(kernel['average_time_per_image']))

            continue
            
    training_conv = []
    evaluating_conv = []
    
    conv_2d = np.transpose(conv_2d)
    
    np.random.shuffle(conv_2d)
    
    conv_2d = np.transpose(conv_2d)

    percentage = 0.8
    length = round(percentage * len(conv_2d[0]))
    
    for column in conv_2d:
        training_conv.append(column[:length])
        evaluating_conv.append(column[length:])

    training_y = np.asarray(training_conv[10])
    evaluating_y = np.asarray(evaluating_conv[10])
    
    training_conv = np.asarray(training_conv)
    evaluating_conv = np.asarray(evaluating_conv)
    
    training_conv = training_conv[:-1]
    evaluating_conv = evaluating_conv[:-1]
    
    print(len(training_y))

4493


In [5]:
    training_conv_time = []
    evaluating_conv_time = []
    
    conv_2d_time = np.transpose(conv_2d_time)
    
    np.random.shuffle(conv_2d_time)
    
    conv_2d_time = np.transpose(conv_2d_time)

    percentage = 0.8
    length = round(percentage * len(conv_2d_time[0]))
    
    for column in conv_2d_time:
        training_conv_time.append(column[:length])
        evaluating_conv_time.append(column[length:])
        
    training_y_time = np.asarray(training_conv_time[10])
    evaluating_y_time = np.asarray(evaluating_conv_time[10])
    
    training_conv_time = np.asarray(training_conv_time)
    evaluating_conv_time = np.asarray(evaluating_conv_time)
    
    training_conv_time = training_conv_time[:-1]
    evaluating_conv_time = evaluating_conv_time[:-1]

In [6]:
    training_conv = np.transpose(training_conv)
    evaluating_conv = np.transpose(evaluating_conv)
    
    print(training_conv[0])
    
    
    reg = LinearRegression()
    reg.fit(training_conv, training_y)

[55. 55. 64. 64.  1.  1.  1. 55. 55. 16.]


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
    training_conv_time = np.transpose(training_conv_time)
    evaluating_conv_time = np.transpose(evaluating_conv_time)
    
    print(training_conv_time[0])
    print(evaluating_conv_time[0])
    
    reg_time = LinearRegression()
    reg_time.fit(training_conv_time, training_y_time)

[55. 55. 64. 64.  1.  1.  1. 55. 55. 16.]
[ 13.  13.  64.  64.   1.   1.   1.  13.  13. 512.]


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
    y_pred = reg.predict(evaluating_conv)
    y_pred_time = reg_time.predict(evaluating_conv_time)

In [9]:
    print('Intercept: \n', reg.intercept_)
    print('Coefficients: \n', reg.coef_)
    
    print('Intercept: \n', reg_time.intercept_)
    print('Coefficients: \n', reg_time.coef_)

Intercept: 
 -0.16753906769355692
Coefficients: 
 [1.14092805e-04 1.14092805e-04 1.10729549e-04 1.10729549e-04
 3.94007735e-02 3.94007735e-02 4.04959176e-06 5.67647082e-04
 5.67647082e-04 1.13184526e-04]
Intercept: 
 -50.12588125550799
Coefficients: 
 [ 1.74575308e-01  1.74575308e-01  2.91105714e-02  2.91105714e-02
  1.15189456e+01  1.15189456e+01 -4.95619498e-04  1.19065924e-01
  1.19065924e-01  4.21807750e-02]


In [10]:
    reg.score(evaluating_conv, evaluating_y)

0.9399009933195297

In [11]:
    reg_time.score(evaluating_conv_time, evaluating_y_time)

0.9178920568760344

In [12]:
    print('Mean Absolute Error:', metrics.mean_absolute_error(evaluating_y, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(evaluating_y, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(evaluating_y, y_pred)))

Mean Absolute Error: 0.024419844840079943
Mean Squared Error: 0.0014090989175044675
Root Mean Squared Error: 0.037537966347479024


In [13]:
    print('Mean Absolute Error:', metrics.mean_absolute_error(evaluating_y_time, y_pred_time))
    print('Mean Squared Error:', metrics.mean_squared_error(evaluating_y_time, y_pred_time))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(evaluating_y_time, y_pred_time)))

Mean Absolute Error: 7.3579104574633005
Mean Squared Error: 125.94249306595847
Root Mean Squared Error: 11.222410305543033


In [14]:
    len(final_test_conv_time)

26

In [15]:
    len(final_test_conv_energy)

26

In [16]:
    print(final_test_conv_time)

[[112, 112, 96, 96, 7, 7, 2, 224, 224, 3], [55, 55, 16, 16, 1, 1, 1, 55, 55, 96], [55, 55, 64, 64, 1, 1, 1, 55, 55, 16], [55, 55, 64, 64, 3, 3, 1, 55, 55, 16], [55, 55, 16, 16, 1, 1, 1, 55, 55, 128], [55, 55, 64, 64, 1, 1, 1, 55, 55, 16], [55, 55, 64, 64, 3, 3, 1, 55, 55, 16], [55, 55, 32, 32, 1, 1, 1, 55, 55, 128], [55, 55, 128, 128, 1, 1, 1, 55, 55, 32], [55, 55, 128, 128, 3, 3, 1, 55, 55, 32], [27, 27, 32, 32, 1, 1, 1, 27, 27, 256], [27, 27, 128, 128, 1, 1, 1, 27, 27, 32], [27, 27, 128, 128, 3, 3, 1, 27, 27, 32], [27, 27, 48, 48, 1, 1, 1, 27, 27, 256], [27, 27, 192, 192, 1, 1, 1, 27, 27, 48], [27, 27, 192, 192, 3, 3, 1, 27, 27, 48], [27, 27, 48, 48, 1, 1, 1, 27, 27, 384], [27, 27, 192, 192, 1, 1, 1, 27, 27, 48], [27, 27, 192, 192, 3, 3, 1, 27, 27, 48], [27, 27, 64, 64, 1, 1, 1, 27, 27, 384], [27, 27, 256, 256, 1, 1, 1, 27, 27, 64], [27, 27, 256, 256, 3, 3, 1, 27, 27, 64], [13, 13, 64, 64, 1, 1, 1, 13, 13, 512], [13, 13, 256, 256, 1, 1, 1, 13, 13, 64], [13, 13, 256, 256, 3, 3, 1, 13,

In [17]:
    print(evaluating_conv_time[0])

[ 13.  13.  64.  64.   1.   1.   1.  13.  13. 512.]


In [18]:
    energy_pred = reg.predict(final_test_conv_energy)
    time_pred = reg_time.predict(final_test_conv_time)

In [19]:
    average_power = sum(energy_pred)/(sum(time_pred)/1000)
    print(average_power)

2.7639669152752786
